In [1]:
from train import train_encoder

/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [2]:
encoder = train_encoder()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 64, 32]               8
       BatchNorm2d-2            [-1, 2, 64, 32]               4
               ELU-3            [-1, 2, 64, 32]               0
            Conv2d-4            [-1, 4, 32, 32]              28
       BatchNorm2d-5            [-1, 4, 32, 32]               8
               ELU-6            [-1, 4, 32, 32]               0
            Conv2d-7            [-1, 8, 32, 16]             104
       BatchNorm2d-8            [-1, 8, 32, 16]              16
               ELU-9            [-1, 8, 32, 16]               0
           Conv2d-10           [-1, 16, 16, 16]             400
      BatchNorm2d-11           [-1, 16, 16, 16]              32
              ELU-12           [-1, 16, 16, 16]               0
           Conv2d-13            [-1, 32, 16, 8]           1,568
      BatchNorm2d-14            [-1, 32

/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/functional.py:515: UserWarning: stft will require the return_complex parameter be explicitly  specified in a future PyTorch release. Use return_complex=False  to preserve the current behavior or return_complex=True to return  a complex output. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:653.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore
/mnt/backup/ml/audio/venv/lib/python3.8/site-packages/torch/functional.py:51

In [3]:
from dataset import AudioSamplePairDataset
from train import transform_sample
test_data = AudioSamplePairDataset(root_path='data/fma_small_samples', test=False, transform=transform_sample)

In [45]:
t = [test_data[i][0] for i in range(1000)]
x = test_data[20][1]

In [46]:
import torch
t_s = torch.stack(t)
x_s = x.unsqueeze(0)

In [47]:
t_e = encoder(t_s.cuda()).cpu()
x_e = encoder(x_s.cuda()).cpu()

In [48]:
dists = (x_e - t_e).abs().sum(dim=-1)

In [51]:
print((x_e - t_e[20]).abs().sum(dim=-1))
print(dists.min())

tensor([10.1731], grad_fn=<SumBackward1>)
tensor(10.1731, grad_fn=<MinBackward1>)
